In [3]:
import pylab
import shutil 
import skimage.morphology as morph
import skimage.feature.texture as texture
import skimage.feature as feature
import skimage.filters as filters
import skimage.transform as transform
import skimage.draw
import skimage.util as utils
from skimage.restoration import denoise_bilateral
from skimage.io import imread
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel,RFE
from sklearn.decomposition import PCA,FastICA
from sklearn.compose import make_column_transformer
from scipy.stats import rv_histogram

In [4]:
root ='/home/marcelo/PV_Cell/' # Caminho para imagens

fnamesF = [] # Nome das imagens PV sem defeito
with open(root+"Sem_Defeito.txt") as f:
 fnamesF = [s.strip() for s in f]

fnamesT = [] # Nome das imagens PV com defeito
with open(root+"Defeito.txt") as f:
 fnamesT =  [s.strip() for s in f]

# Abre arquivos de imagens em coloca em listas separadas
imF = [imread(root+"Sem_Defeito/"+f) for f in fnamesF]
imT = [imread(root+"Defeito/"+f) for f in fnamesT]

im = {-1:imF,1:imT}

roi_1 = [(10,25),(43,265)]
roi_2 = [(60,15),(140,280)]
roi_3 = [(156,15),(238,280)]
roi_4 = [(255,25),(290,265)]

In [214]:

def glsm_features(img):
  g = texture.greycomatrix(utils.img_as_ubyte(img), [1,2,3], [0,pylab.pi/4,pylab.pi/2,3*pylab.pi/4], 
                           levels=256, normed=True, symmetric=True)
  l = []
  for f in ['contrast','dissimilarity','homogeneity','ASM','energy','correlation']:
    l.append(texture.greycoprops(g,f).std())
  return l

def glsm_features2(img):
  offsets = [1,3,5,7]
  angles =  pylab.linspace(0,2*pylab.pi,16)
  g = texture.greycomatrix(utils.img_as_ubyte(img),offsets,angles, 
                           levels=256, normed=False, symmetric=False)
  aux = []
  for f in ['contrast','dissimilarity','homogeneity','energy','correlation']:
   #aux.append(texture.greycoprops(g,f).flatten())
   aux = aux + texture.greycoprops(g,f).ravel().tolist()
  return aux  
  #return PCA(whiten=False).fit_transform(pylab.array(aux))

def LBP_features(img,radius,method):
 h = []   
 for r in radius:
  lbp = feature.local_binary_pattern(img, 8 * r, r, method)
  n_bins = int(lbp.max() + 1)
  h.append(pylab.histogram(lbp, density=True, bins=n_bins, range=(0, n_bins)))
 return h

 

In [221]:
X1 =[]
X2 = []
lbl = []
i = 0
for label,img_list in im.items():
    for img in img_list:  
     i = i+1  
     print(i,' ',end='')
     aux1,aux2 = [],[] 
     for roi in [roi_1,roi_2,roi_3,roi_4]: 
      x0,y0 = roi[0]
      x1,y1 = roi[1]
      aux1 = aux1 + LBP_features(img[x0:x1,y0:y1],[1,2,3],"uniform")
      aux2 = aux2 + glsm_features2(img[x0:x1,y0:y1])
     lbl.append(label)
     X1.append(aux1)
     X2.append(aux2)
     
     
      

1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  218  219  220  221  222

In [226]:
#x = [i.flatten().tolist()+j.flatten().tolist()+k.flatten().tolist()+l.flatten().tolist() for i,j,k,l in zip(X[0],X[1],X[2],X[3])]
x1 = []
for x in X1:
 aux = []
 for h in x:
   rv = rv_histogram(h)
   st = pylab.hstack((rv.stats(moments='mvsk'),rv.entropy()))
   aux = aux + st.tolist()
 x1.append(aux)
x1 = pylab.array(x1)
x2 = pylab.array(X2)

In [287]:
clf1 = RandomForestClassifier(n_estimators=250,min_samples_split=3,n_jobs=-1)
clf2 = SVC(kernel='linear',probability=True)
clf3 = LogisticRegression(max_iter=350)
clf4 = GaussianNB()
clf5 = KNeighborsClassifier(n_neighbors = 3)
clf6 = VotingClassifier(estimators=[('rfc',clf1),('svc',clf2),('lr',clf3),('nb',clf4),('knn',clf5)],voting='soft')

In [ ]:
xf,y = pylab.hstack((x2,x1)),lbl
N,N1,N2 = xf.shape[1],x1.shape[1],x2.shape[1]
k = 5
for n in range(5,N2,1):   
  print(n,end='')
  #slm = SelectFromModel(estimator=clf1,max_features=n,prefit=False)
  ctf = make_column_transformer((PCA(n_components=n,whiten=True),slice(0,N2)),remainder="passthrough")
  for clf in [clf1,clf2,clf3,clf4,clf5,clf6]:
   p2 = Pipeline([('Scaler',StandardScaler(with_mean=False)),('ctf',ctf),("Classification",clf)])
   score = cross_val_score(p2,xf,y, cv=StratifiedKFold(n_splits=3,shuffle=True),scoring='f1',n_jobs=-1)
   print(" {0:.2f} {1:.2f}".format(score.mean(),score.std()),end='')
  print()

5 0.86 0.02 0.86 0.03 0.87 0.03 0.86 0.02 0.85 0.01 0.86 0.03
6 0.86 0.01 0.87 0.00 0.86 0.02 0.86 0.02 0.82 0.02 0.86 0.03
7 0.87 0.01 0.86 0.02 0.87 0.02 0.86 0.01 0.84 0.02 0.86 0.03
8 0.86 0.02 0.87 0.02 0.85 0.03 0.86 0.01 0.84 0.03 0.87 0.01
9 0.86 0.02 0.87 0.04 0.87 0.01 0.86 0.01 0.86 0.02 0.87 0.02
10 0.87 0.03 0.88 0.02 0.89 0.04 0.86 0.01 0.85 0.02 0.87 0.00
11 0.87 0.03 0.88 0.01 0.88 0.01 0.86 0.01 0.83 0.02 0.87 0.02
12 0.86 0.02 0.88 0.02 0.86 0.04 0.86 0.01 0.84 0.04 0.87 0.03
13 0.87 0.03 0.89 0.01 0.88 0.01 0.86 0.01 0.84 0.02 0.87 0.02
14 0.87 0.01 0.89 0.03 0.89 0.02 0.86 0.04 0.83 0.00 0.87 0.03
15 0.87 0.01 0.87 0.02 0.88 0.01 0.86 0.03 0.82 0.02 0.87 0.02
16 0.87 0.03 0.85 0.02 0.86 0.02 0.86 0.01 0.83 0.02 0.87 0.04
17 0.86 0.02 0.88 0.01 0.88 0.03 0.86 0.02 0.83 0.01 0.87 0.01
18 0.88 0.04 0.89 0.02 0.88 0.02 0.86 0.03 0.83 0.01 0.87 0.02
19 0.87 0.01 0.87 0.03 0.87 0.03 0.86 0.03 0.84 0.01 0.87 0.02
20 0.87 0.03 0.87 0.04 0.87 0.03 0.86 0.01 0.84 0.03 0.87 0.

In [ ]:
img = imT[10]
imr = imread(root+"cell_msk.png",as_gray=True)

#imr_g = filters.gaussian(imr,sigma = 2.) 
#img_g = filters.gaussian(img,sigma = .5) 
img_g =denoise_bilateral(img,sigma_spatial = 5.) 
#imr_t = imr_g > filters.threshold_otsu(imr_g)
img_t = img_g > filters.threshold_otsu(img_g)

msk = pylab.logical_xor(imr,img_t)
#msk = morph.binary_opening(msk,morph.rectangle(20,5)) 
#msk = morph.binary_closing(msk,morph.disk(5))    
#

pylab.figure(i,figsize=(10,20))
pylab.subplot(121)
pylab.imshow(img,'gray')
pylab.subplot(122)
pylab.imshow(img_t, 'gray')
 